In [3]:
"""
Me robé los imports de otra función, después debería emprolijar borrando las que no use
"""

import argparse
import os
import os.path
import shutil
from datetime import datetime as dt
import logging
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import cryodrgn
from cryodrgn import analysis, utils, config

In [ ]:
#Levanto los centros de KMeans en UMAP del modelo original (downs=256, z=8)



In [ ]:
logger = logging.getLogger(__name__)

t1 = dt.now()
#E = args.epoch
E = 64
#workdir = args.workdir
workdir = '/nfs/bartesaghilab2/ds672/empiar10076/2023_11_16_output_tutorial_downsampling_256_z_8_myposes_iteration2_filtered_refinament/'
zfile = f"{workdir}/z.{E}.pkl"
weights = f"{workdir}/weights.{E}.pkl"
cfg = (
    f"{workdir}/config.yaml"
    if os.path.exists(f"{workdir}/config.yaml")
    else f"{workdir}/config.pkl"
)
outdir = f"{workdir}/analyze.{E}"
if E == -1:
    zfile = f"{workdir}/z.pkl"
    weights = f"{workdir}/weights.pkl"
    outdir = f"{workdir}/analyze"

# if args.outdir:
#     outdir = args.outdir
outdir = os.path.join(workdir, 'analyze.64/testing_volumes_per_indexs')
logger.info(f"Saving results to {outdir}")
if not os.path.exists(outdir):
    os.mkdir(outdir)

z = utils.load_pkl(zfile)
zdim = z.shape[1]

# vol_args = dict(
#     Apix=args.Apix,
#     downsample=args.downsample,
#     flip=args.flip,
#     device=args.device,
#     invert=args.invert,
#     vol_start_index=args.vol_start_index,
# )

vol_args = dict(
    Apix=3.275,
    downsample=None,
    flip=args.flip,
    device=None,
    invert=args.invert,
    vol_start_index=None,
)
vg = VolumeGenerator(weights, cfg, vol_args, skip_vol=args.skip_vol)

In [ ]:
zdim = z.shape[1]

# Principal component analysis
logger.info("Performing principal component analysis...")
pc, pca = analysis.run_pca(z)
logger.info("Generating volumes for trajectories analysis...")
for i in range(num_pcs):
    start, end = np.percentile(pc[:, i], (5, 95))
    z_pc = analysis.get_pc_traj(pca, z.shape[1], 10, i + 1, start, end)
    vg.gen_volumes(f"{outdir}/pc{i+1}", z_pc)

# kmeans clustering
logger.info("K-means clustering...")
K = num_ksamples
kmeans_labels, centers = analysis.cluster_kmeans(z, K)
centers, centers_ind = analysis.get_nearest_point(z, centers)
if not os.path.exists(f"{outdir}/kmeans{K}"):
    os.mkdir(f"{outdir}/kmeans{K}")
utils.save_pkl(kmeans_labels, f"{outdir}/kmeans{K}/labels.pkl")
np.savetxt(f"{outdir}/kmeans{K}/centers.txt", centers)
np.savetxt(f"{outdir}/kmeans{K}/centers_ind.txt", centers_ind, fmt="%d")
logger.info("Generating volumes...")
vg.gen_volumes(f"{outdir}/kmeans{K}", centers)